In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../data/'
folder = 'sp_matrix_pop'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site

In [2]:
df = pd.read_pickle(os.path.join(dir_, folder, file_name))
df_train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name[:-3] + 'pkl'))
train_y = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')

num_users = len(df['uid'].unique())
num_tracks = len(df['tid'].unique())
print(num_users, num_tracks)

218 39137


In [3]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 1
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 218
Tag start position : 39355
Length of vector : 39356
----------------------------------------
Number of unique users : 218
Number of unique tracks : 39137


In [4]:
pop = {}
for i in df['tid'].unique():
    pop[i] = len(df[df['tid']==i])/unique_users

In [5]:
# pop
i = 0
v = []
sp_rows = []
sp_cols = []
for index, row in tqdm(df_train.iterrows(), total=len(df_train)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    v.append(1)

    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row['tid']])
        
    i += 1

train_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(df_train), l), dtype=float)

In [6]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [7]:
x_test = []
for i in tqdm(range(unique_users)):
    user = df_train[df_train['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)

In [8]:
i = 0
v = []
sp_rows = []
sp_cols = []
for row in tqdm(x_test):
    sp_rows.append(i)
    sp_cols.append(row[0])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row[1]])
    
    i += 1

In [9]:
test_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(x_test), l), dtype=float)

In [10]:
test_x

<8169189x39356 sparse matrix of type '<class 'numpy.float64'>'
	with 24507567 stored elements in Compressed Sparse Row format>

In [11]:
print(test_x[0,:])

  (0, 0)	1.0
  (0, 220)	1.0
  (0, 39355)	0.11009174311926606


In [24]:
order = 3
model = TFFMRegressor(
    order=order, 
    rank=40, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.00001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)
predictions = model.predict(test_x)

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
100%|██████████| 10/10 [00:19<00:00,  1.98s/epoch]


In [25]:
len(predictions), predictions

(8169189,
 array([0.09527917, 0.06645159, 0.12292972, ..., 0.07455038, 0.07124534,
        0.07223253], dtype=float32))

In [26]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', predictions, False) 
df2[:10]

,uid,tid,rating
0,0,2,0.095279
1,0,14,0.066452
2,0,15,0.122930
3,0,20,0.083724
4,0,21,0.069101
5,0,24,0.094925
6,0,30,0.066488
7,0,44,0.075515
8,0,46,0.076777
9,0,48,0.149123


In [31]:
df2.to_pickle(os.path.join('../data/sp_matrix_pop(tag,new)', 'topN_pred_' + file_name[:-3] + 'pkl'))

In [28]:
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [29]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(unique_users)):
    prediction = df2[df2['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(unique_users)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/unique_users)


0.15596330275229367
